In [ ]:

import time
import pandas as pd
import urllib.request
import requests
from bs4 import BeautifulSoup
import re

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter                import PorterStemmer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stopwords_eng = set(stopwords.words("english"))
snowball = SnowballStemmer(language="english")
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers
from sklearn.metrics import accuracy_score, classification_report
import scikitplot as skplt

from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.preprocessing.text import Tokenizer

import tensorflow
from tensorflow import keras
#import torchtext
import matplotlib.pyplot as plt
%matplotlib inline
import gc
#!pip install pycorenlp
from pycorenlp import StanfordCoreNLP
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('https://calm-panther-70.loca.lt')
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim import models
from gensim.models.keyedvectors import KeyedVectors
import gensim.downloader as api

#!pip install wget
import wget
import gzip

# url = "https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing"

wv = api.load('word2vec-google-news-300')

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense
from keras import backend as K 
from sklearn.metrics import accuracy_score, classification_report

from sklearn.metrics import confusion_matrix
import scikitplot as skplt
import matplotlib.pyplot as plt
from lime import lime_text


#### setting all seeds for reproducible results
seed_value= 420
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.random.set_seed(seed_value)

In [ ]:
#ff: fast forward to this point
merged_pdf_df = pd.read_csv ('merged_pdf_df_clean.csv')

In [ ]:
NUM_WORDS=1500

sentences = merged_pdf_df.clean_content

tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
word_vectors = wv

EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [ ]:
embedding_matrix

In [ ]:
words = pd.DataFrame(word_index.items(), columns=['word', 'value'])
words

In [ ]:
from sklearn.decomposition import PCA
from matplotlib import pyplot

# fit a 2d PCA model to the vectors
X = embedding_matrix
pca = PCA(n_components=2)
result = pca.fit_transform(X)

# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(words['word'])[1:300]
for i, word in enumerate(words):
	pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

## Split data into training and test

In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import regularizers

In [ ]:
#the above three are from source
# OR run this one
rs = 420
maxlen = 500 #state how long the sequences should be - anything longer that this gets cut
word_num = 500
#embedding_dim = 50
embedding_dim = 128
x_y_xplit = 0.25


train_dataset = merged_pdf_df.sample(frac=0.8,random_state=50) #random state is a seed value
test_dataset  = merged_pdf_df.drop(train_dataset.index)

X_test_text = list(test_dataset['clean_text_tok'])
Y_test = list(test_dataset['pnn_class'])
X_train_text = list(train_dataset['clean_text_tok'])
Y_train = list(train_dataset['pnn_class'])

unique_classes = list(set(Y_train))
target_classes = ["UP", "DOWN", "SAME"]

from sklearn.preprocessing import LabelEncoder
labenc = LabelEncoder()
Y_train = labenc.fit_transform(Y_train)
Y_test = labenc.fit_transform(Y_test)

## Subtracted 1 from labels to bring range from 1-4 to 0-3
#Y_train, Y_test = np.array(Y_train), np.array(Y_test)

In [ ]:
tokenizer = Tokenizer(num_words=word_num)
tokenizer.fit_on_texts(X_train_text)

X_train = tokenizer.texts_to_sequences(X_train_text)
X_test = tokenizer.texts_to_sequences(X_test_text)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [ ]:
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen) #post padding so 0s get added to the end, not start
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0, :])

In [ ]:
#function to plot results
import matplotlib.pyplot as plt

def plot_loss_and_acc(history):
    #train_loss = history.history["loss"]
    #train_acc = history.history["accuracy"]
    #val_loss = history.history["val_loss"]
    #val_acc = history.history["val_accuracy"]

    history_dict = history.history
    print(history_dict.keys())

    train_acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    train_loss = history_dict['loss']
    val_loss = history_dict['val_loss']    

    
    fig = plt.figure(figsize=(12,5))

    ax = fig.add_subplot(121)
    ax.plot(range(len(train_loss)), train_loss, label="Train Loss");
    ax.plot(range(len(val_loss)), val_loss, label="Validation Loss");
    plt.xlabel("Epochs"); plt.ylabel("Loss");
    plt.title("Train Loss vs Validation Loss");
    plt.legend(loc="best");

    ax = fig.add_subplot(122)
    ax.plot(range(len(train_acc)), train_acc, label="Train Accuracy");
    ax.plot(range(len(val_acc)), val_acc, label="Validation Accuracy");
    plt.xlabel("Epochs"); plt.ylabel("Accuracy");
    plt.title("Train Accuracy vs Validation Accuracy");
    plt.legend(loc="best");  

### Model - CNN

In [ ]:
sequence_length = X_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

Y_train_cat = to_categorical(Y_train, 3)
Y_test_cat = to_categorical(Y_test, 3)

#unique_classes = list(set(Y_train))
target_classes = ["UP", "DOWN", "SAME"]

inputs    = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape   = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(len(target_classes), activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)

In [ ]:
adam = Adam(learning_rate=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
es = EarlyStopping(monitor="val_loss", mode="auto",patience=3, verbose=1)

history = model.fit(X_train, Y_train_cat, batch_size=500, epochs=16
                    , validation_data=(X_test, Y_test_cat),callbacks=[es])

loss, accuracy = model.evaluate(X_train, Y_train_cat, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test_cat, verbose=True)
print("Testing Accuracy:  {:.4f}".format(accuracy))

train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print("Train Accuracy : {}".format(accuracy_score(Y_train, np.argmax(train_preds, axis=1))))
print("Test  Accuracy : {}".format(accuracy_score(Y_test, np.argmax(test_preds, axis=1))))
print("\nClassification Report : ")
print(classification_report(Y_test, np.argmax(test_preds, axis=1), target_names=target_classes))


skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_test], [target_classes[i] for i in np.argmax(test_preds, axis=1)],
                                    normalize=False,
                                    title="Confusion Matrix",
                                    cmap="Greens",
                                    hide_zeros=True,
                                    figsize=(8,8)
                                    );
plt.xticks(rotation=90);

gc.collect()
K.clear_session() #this clears the session to forget previous model executions and become overfitted
#model.fit(X_train_inp, y_train_inp, batch_size=1000, epochs=8, verbose=1, validation_data=(X_val_inp, y_val_inp))  # starts training

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


es = EarlyStopping(monitor="val_loss", mode='auto',patience=3, verbose=1)

history = model.fit(X_train, Y_train_cat,
                    epochs=32,
                    verbose=False,
                    validation_data=(X_test, Y_test_cat),
                    batch_size=400, 
                    callbacks=[es])
    
loss, accuracy = model.evaluate(X_train, Y_train_cat, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test_cat, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_loss_and_acc(history)  


train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print("Train Accuracy : {}".format(accuracy_score(Y_train, np.argmax(train_preds, axis=1))))
print("Test  Accuracy : {}".format(accuracy_score(Y_test, np.argmax(test_preds, axis=1))))
print("\nClassification Report : ")
print(classification_report(Y_test, np.argmax(test_preds, axis=1), target_names=target_classes))


skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_test], [target_classes[i] for i in np.argmax(test_preds, axis=1)],
                                    normalize=False,
                                    title="2D CNN Confusion Matrix",
                                    cmap="Reds",
                                    hide_zeros=True,
                                    figsize=(8,8)
                                    );
plt.xticks(rotation=90);

K.clear_session() #this clears the session to forget previous model executions and become overfitted

In [ ]:
loss, accuracy = model.evaluate(X_train, Y_train_cat, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test_cat, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_loss_and_acc(history)  


train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print("Train Accuracy : {}".format(accuracy_score(Y_train, np.argmax(train_preds, axis=1))))
print("Test  Accuracy : {}".format(accuracy_score(Y_test, np.argmax(test_preds, axis=1))))
print("\nClassification Report : ")
print(classification_report(Y_test, np.argmax(test_preds, axis=1), target_names=target_classes))


skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_test], [target_classes[i] for i in np.argmax(test_preds, axis=1)],
                                    normalize=False,
                                    title="2D CNN Confusion Matrix",
                                    cmap="Reds",
                                    hide_zeros=True,
                                    figsize=(8,8)
                                    );
plt.xticks(rotation=90);

K.clear_session() #this clears the session to forget previous model executions and become overfitted

In [ ]:
plot_loss_and_acc(history)  

print("Train Accuracy : {}".format(accuracy_score(Y_train, np.argmax(train_preds, axis=1))))
print("Test  Accuracy : {}".format(accuracy_score(Y_test, np.argmax(test_preds, axis=1))))
print("\nClassification Report : ")
print(classification_report(Y_test, np.argmax(test_preds, axis=1), target_names=target_classes))


skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_test], [target_classes[i] for i in np.argmax(test_preds, axis=1)],
                                    normalize=False,
                                    title="Confusion Matrix",
                                    cmap="Reds",
                                    hide_zeros=True,
                                    figsize=(8,8)
                                    );
plt.xticks(rotation=90);

gc.collect()
K.clear_session() #this clears the session to forget previous model executions and become overfitted
#model.fit(X_train_inp, y_train_inp, batch_size=1000, epochs=8, verbose=1, validation_data=(X_val_inp, y_val_inp))  # starts training

In [ ]:
history

In [ ]:
model.metrics_names

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

print(val_loss)

In [ ]:
loss, accuracy = model.evaluate(X_train_inp, y_train_inp, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_val_inp, y_val_inp, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_loss_and_acc(history)  


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense
from keras import backend as K 
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
train_preds = model.predict(X_train_inp)
test_preds = model.predict(X_val_inp)

In [ ]:


print(classification_report(y_val_inp, np.argmax(test_preds, axis=1), target_names=target_classes))


skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_test], [target_classes[i] for i in np.argmax(test_preds, axis=1)],
                                    normalize=False,
                                    title="Confusion Matrix",
                                    cmap="Greens",
                                    hide_zeros=True,
                                    figsize=(8,8)
                                    );
plt.xticks(rotation=90);

K.clear_session() #this clears the session to forget previous model executions and become overfitted

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_and_acc(history):
    train_loss = history.history["loss"]
    train_acc = history.history["acc"]
    val_loss = history.history["val_loss"]
    val_acc = history.history["val_acc"]

    fig = plt.figure(figsize=(12,5))

    ax = fig.add_subplot(121)
    ax.plot(range(len(train_loss)), train_loss, label="Train Loss");
    ax.plot(range(len(val_loss)), val_loss, label="Validation Loss");
    plt.xlabel("Epochs"); plt.ylabel("Loss");
    plt.title("Train Loss vs Validation Loss");
    plt.legend(loc="best");

    ax = fig.add_subplot(122)
    ax.plot(range(len(train_acc)), train_acc, label="Train Accuracy");
    ax.plot(range(len(val_acc)), val_acc, label="Validation Accuracy");
    plt.xlabel("Epochs"); plt.ylabel("Accuracy");
    plt.title("Train Accuracy vs Validation Accuracy");
    plt.legend(loc="best");

plot_loss_and_acc(history)

### NOW DO LIME

In [ ]:
from lime import lime_text

explainer = lime_text.LimeTextExplainer(class_names=target_classes, verbose=True)

def make_predictions(X_batch_text):
    X_batch = pad_sequences(tokenizer.texts_to_sequences(X_batch_text), maxlen=500, padding="post", truncating="post", value=0)
    preds = model.predict(X_batch)
    return preds

def lime_run(i):
    rng = np.random.RandomState(i)
    idx = rng.randint(1, len(X_test))

    print("Prediction : ", target_classes[model.predict(X_test[idx:idx+1]).argmax(axis=-1)[0]])
    print("Actual :     ", target_classes[Y_test[idx]])

    explanation = explainer.explain_instance(X_test_text[idx], classifier_fn=make_predictions, labels=Y_test[idx:idx+1], num_features=15)
    explanation.show_in_notebook()

In [ ]:
for i in range(400,450):
    lime_run(i)